<a href="https://colab.research.google.com/github/nikbizkit/MMO/blob/main/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances

Загрузим датасет:

In [6]:
df_perf_all = pd.read_csv('laptop_data.csv', sep=",")
df_perf_all.head()


,brand,laptop_name,display_size,processor_type,graphics_card,disk_space,discount_price,old_price,ratings_5max
0,HP,Notebook 14-df0008nx,14.0,Intel Celeron N4000,Intel HD Graphics 600,64 GB (eMMC),1259.0,1259.0,0 / 5
1,Lenovo,IdeaPad 330S-14IKB,14.0,Intel Core i5-8250U,Intel UHD Graphics 620,1 TB HDD,1849.0,2099.0,3.3 / 5
2,Huawei,MateBook D Volta,14.0,Intel Core i5-8250U,NVIDIA GeForce MX150 (2 GB),256 GB SSD,2999.0,3799.0,0 / 5
3,Dell,Inspiron 15 3567,15.6,Intel Core i3-7020U,Intel HD Graphics 620,1 TB HDD,1849.0,1849.0,0 / 5
4,Asus,VivoBook 15 X510UR,15.6,Intel Core i7-8550U,NVIDIA GeForce 930MX (2 GB),1 TB HDD,2499.0,3149.0,0 / 5


In [7]:
df_perf_all.shape

(205, 9)

Убедимся, что в нашем рабочем датафрейме не будет записей с отсутсвующей видеокартой: 

In [12]:
df_perf_with_graphics_card = df_perf_all[df_perf_all['graphics_card'].notnull()]
df_perf_with_graphics_card = df_perf_with_graphics_card[~df_perf_with_graphics_card['graphics_card'].str.isspace()]

In [13]:
brand = df_perf_with_graphics_card['brand'].values
brand[0:6]

array(['HP', 'Lenovo', 'Huawei', 'Dell', 'Asus', 'Dell'], dtype=object)

In [15]:
processor_type= df_perf_with_graphics_card['processor_type'].values
processor_type[0:5]

array([' Intel Celeron N4000', ' Intel Core i5-8250U',
       ' Intel Core i5-8250U', ' Intel Core i3-7020U',
       ' Intel Core i7-8550U'], dtype=object)

In [16]:
graphics_card = df_perf_with_graphics_card['graphics_card'].values
graphics_card[0:3]

array([' Intel HD Graphics 600', ' Intel UHD Graphics 620',
       ' NVIDIA GeForce MX150 (2 GB)'], dtype=object)

Векторизуем видеокарту с помощью Tf-Idf Vectorizer

In [17]:
tfidfv = TfidfVectorizer()
descr_matrix = tfidfv.fit_transform(graphics_card)
descr_matrix

<205x57 sparse matrix of type '<class 'numpy.float64'>'
	with 837 stored elements in Compressed Sparse Row format>

И с помощью CountVectorizer:

In [18]:
countv = CountVectorizer()
graphics_card_matrix_co = countv.fit_transform(graphics_card)
graphics_card_matrix_co

<205x57 sparse matrix of type '<class 'numpy.int64'>'
	with 837 stored elements in Compressed Sparse Row format>

In [19]:
class SimpleKNNRecommender:
    
    def __init__(self, X_matrix, X_names, X_notes, X_descr):
        """
        Входные параметры:
        X_matrix - обучающая выборка (матрица объект-признак)
        X_ids - массив идентификаторов объектов
        X_title - массив названий объектов
        X_overview - массив описаний объектов
        """
        #Сохраняем параметры в переменных объекта
        self._X_matrix = X_matrix
        self.df = pd.DataFrame(
            {'brand': pd.Series(X_names, dtype='str'),
            'processor_type': pd.Series(X_notes, dtype='str'),
            'graphics_card': pd.Series(X_descr, dtype='str'),
            'Dist': pd.Series([], dtype='float')})
            
            
    def recommend_for_single_object(self, K: int, \
                X_matrix_object, cos_flag = True, manh_flag = False):
        """
        Метод формирования рекомендаций для одного объекта.
        Входные параметры:
        K - количество рекомендуемых соседей 
        X_matrix_object - строка матрицы объект-признак, соответствующая объекту
        cos_flag - флаг вычисления косинусного расстояния
        manh_flag - флаг вычисления манхэттэнского расстояния
        Возвращаемое значение: K найденных соседей
        """
        
        scale = 1000000
        # Вычисляем косинусную близость
        if cos_flag:
            dist = cosine_similarity(self._X_matrix, X_matrix_object)
            self.df['Dist'] = dist * scale
            res = self.df.sort_values(by='Dist', ascending=False)
            # Не учитываем рекомендации с единичным расстоянием,
            # так как это искомый объект
            res = res[res['Dist'] < scale]
        
        else:
            if manh_flag:
                dist = manhattan_distances(self._X_matrix, X_matrix_object)
            else:
                dist = euclidean_distances(self._X_matrix, X_matrix_object)
            self.df['Dist'] = dist * scale
            res = self.df.sort_values(by='Dist', ascending=True)
            # Не учитываем рекомендации с единичным расстоянием,
            # так как это искомый объект
            res = res[res['Dist'] > 0.0]            
        
        # Оставляем К первых рекомендаций
        res = res.head(K)
        return res

Выберем тестовый образец, на основе которого мы будем давать рекомендации:

In [23]:
test_name = 7
brand[test_name]

'Huawei'

Зададим его матрицу: 

In [24]:
test_matrix = descr_matrix[test_name]
test_matrix

<1x57 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [25]:
skr1 = SimpleKNNRecommender(descr_matrix, brand, processor_type, graphics_card)

In [26]:
test = df_perf_with_graphics_card.iloc[test_name]
test

brand                                   Huawei
laptop_name                         MateBook D
display_size                              15.6
processor_type             Intel Core i5-8250U
graphics_card      NVIDIA GeForce MX150 (2 GB)
disk_space                 128 GB SSD/1 TB HDD
discount_price                          2449.0
old_price                               2799.0
ratings_5max                           4.4 / 5
Name: 7, dtype: object

Делаем рекомендацию на основании видеокарты векторизованного Tf-Idf и косинусного  расстояния:

In [27]:
rec1 = skr1.recommend_for_single_object(15, test_matrix)
rec1

,brand,processor_type,graphics_card,Dist
148,Asus,Intel Core i7-8550U,NVIDIA GeForce MX130 (2 GB),521728.255476
151,HP,Intel Core i5-8250U,NVIDIA GeForce MX130 (2 GB),521728.255476
48,HP,Intel Core i7-8565U,NVIDIA GeForce MX130 (4 GB),521728.255476
49,HP,Intel Core i7-8565U,NVIDIA GeForce MX130 (4 GB),521728.255476
51,Asus,Intel Core i7-8550U,NVIDIA GeForce MX130 (2 GB),521728.255476
87,Dell,Intel Core i7-8565U,NVIDIA GeForce MX130 (4 GB),521728.255476
53,Acer,Intel Core i7-8565U,NVIDIA GeForce MX130 (2 GB),521728.255476
150,HP,Intel Core i5-8250U,NVIDIA GeForce MX130 (2 GB),521728.255476
58,HP,Intel Core i5-8265U,NVIDIA GeForce MX130 (2 GB),521728.255476
183,Acer,Intel Core i7-8550U,NVIDIA GeForce MX130 (2 GB),521728.255476


In [37]:
test_matrix_co = graphics_card_matrix_co[test_name]
test_matrix_co

<1x57 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [38]:
skr2 = SimpleKNNRecommender(graphics_card_matrix_co, brand, processor_type, graphics_card)

Делаем рекомендации по видеокартам векторизованным CountVectorizer и на основе Евклидова расстояния:

In [39]:
rec2 = skr2.recommend_for_single_object(15, graphics_card_matrix_co, cos_flag = False)
rec2

,brand,processor_type,graphics_card,Dist
85,Dell,Intel Celeron N4000,Intel UHD Graphics 600,1.414214e+06
73,Lenovo,Intel Celeron N4000,Intel UHD Graphics 600,1.414214e+06
69,Dell,Intel Core i3-7020U,Intel HD Graphics 620,1.414214e+06
109,Microsoft,Intel Core i5-7200U,Intel HD Graphics 620,1.414214e+06
108,Acer,Intel Celeron N3350,Intel HD Graphics 505,1.414214e+06
42,Apple,Intel Core M3,Intel HD Graphics 615,1.414214e+06
129,HP,Intel Celeron N4000,Intel UHD Graphics 600,1.414214e+06
142,HP,Intel Core i7-7500U,Intel HD Graphics 620,1.414214e+06
106,HP,Intel Celeron N3060,Intel HD Graphics 400,1.414214e+06
146,Apple,Intel Core M3,Intel HD Graphics 615,1.414214e+06
